In [1]:
import pandas as pd
import numpy as np

from xgboost import XGBRegressor
# from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterBaseline import ForecasterEquivalentDate
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection import bayesian_search_forecaster

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
import plotly.offline as poff
import seaborn as sns

pio.renderers.default = 'notebook' 
pio.templates.default = "seaborn"
poff.init_notebook_mode(connected=True)
plt.style.use('seaborn-v0_8-darkgrid')

In [2]:
# Read data
# ==============================================================================
data_dir = 'Data/spain/four years'
# data_dir = 'Data/spain/one year'

df = pd.read_csv(data_dir + '/spain_clean.csv')
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index(keys = 'datetime', inplace=True)
df = df.asfreq('60min')
df.head()

,wave_height,period
datetime,,
2020-06-18 00:00:00,0.78,4.92
2020-06-18 01:00:00,0.76,5.06
2020-06-18 02:00:00,0.74,5.16
2020-06-18 03:00:00,0.73,5.22
2020-06-18 04:00:00,0.73,5.28


In [3]:
# Train-test split
# ==============================================================================
one_month = (-1)*30*24 # One month
two_months = (-1)*60*24 # Two months

end_train = two_months + two_months
end_val = two_months

df_train = df.iloc[:end_train].copy()
df_val = df.iloc[end_train:end_val].copy()
df_test = df.iloc[end_val:].copy()

print(f"Train dates      : {df_train.index.min()} --- {df_train.index.max()}  (n={len(df_train)})")
print(f"Validation dates : {df_val.index.min()} --- {df_val.index.max()}  (n={len(df_val)})")
print(f"Test dates       : {df_test.index.min()} --- {df_test.index.max()}  (n={len(df_test)})")

Train dates      : 2020-06-18 00:00:00 --- 2024-02-19 23:00:00  (n=32208)
Validation dates : 2024-02-20 00:00:00 --- 2024-04-19 23:00:00  (n=1440)
Test dates       : 2024-04-20 00:00:00 --- 2024-06-18 23:00:00  (n=1440)


## Baseline model

In [4]:
# Create baseline: value of the same hour of the previous day
# ==============================================================================
forecaster = ForecasterEquivalentDate(
                 offset    = pd.DateOffset(hours=1),
                 n_offsets = 1
             )

# Train forecaster
# ==============================================================================
forecaster.fit(y=df.iloc[:end_val]['wave_height'])
forecaster

ForecasterEquivalentDate 
Offset: <DateOffset: hours=1> 
Number of offsets: 1 
Aggregation function: mean 
Window size: 1 
Training range: [Timestamp('2020-06-18 00:00:00'), Timestamp('2024-04-19 23:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: 60T 
Creation date: 2024-06-26 10:08:54 
Last fit date: 2024-06-26 10:08:54 
Skforecast version: 0.12.1 
Python version: 3.11.9 
Forecaster id: None 

In [5]:
# Backtesting
# ==============================================================================
metric, predictions = backtesting_forecaster(
                          forecaster         = forecaster,
                          y                  = df['wave_height'],
                          steps              = 24,
                          metric             = 'mean_absolute_error',
                          initial_train_size = len(df.iloc[:end_val]),
                          refit              = False,
                          n_jobs             = 'auto',
                          verbose            = False,
                          show_progress      = True
                      )

print(f"Backtest error (MAE): {metric}")

  0%|          | 0/60 [00:00<?, ?it/s]

Backtest error (MAE): 0.19531944444444443


## Recursive multi-step forecasting

A recursive autoregressive model `ForecasterAutoreg` is trained using a gradient boosting regressor `XGBRegressor` as the base regressor. A time window of 24 hours (24 lags) is used to predict the next hour's demand. This means that the demand values of the previous 24 hours are used as predictors. The hyperparameters of the underlying regressor are left at their default values.

We add a custom weight function to ignore missing values during training. 



In [6]:
# Create forecaster
# ==============================================================================
forecaster = ForecasterAutoreg(
                regressor = XGBRegressor(random_state = 42),
                lags = 24)

# Train forecaster
# ==============================================================================
forecaster.fit(y=df.iloc[:end_val]['wave_height'])
forecaster

ForecasterAutoreg 
Regressor: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...) 
Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24] 
Transformer for y: None 
Transformer for exog: None 
Window size: 24 
Weight function included: False 
Differentiation order: None 
Exogenous i

In [7]:
# Backtesting
# ==============================================================================
metric, predictions = backtesting_forecaster(
                          forecaster         = forecaster,
                          y                  = df['wave_height'],
                          steps              = 24,
                          metric             = 'mean_absolute_error',
                          initial_train_size = len(df.iloc[:end_val]),
                          refit              = False,
                          n_jobs             = 'auto',
                          verbose            = True,
                          show_progress      = True
                      )

# Backtesting error
# ==============================================================================
print(f'Backtest error (MAE): {metric}')

Information of backtesting process
----------------------------------
Number of observations used for initial training: 33648
Number of observations used for backtesting: 1440
    Number of folds: 60
    Number of steps per fold: 24
    Number of steps to exclude from the end of each train set before test (gap): 0

Fold: 0
    Training:   2020-06-18 00:00:00 -- 2024-04-19 23:00:00  (n=33648)
    Validation: 2024-04-20 00:00:00 -- 2024-04-20 23:00:00  (n=24)
Fold: 1
    Training:   2020-06-18 00:00:00 -- 2024-04-19 23:00:00  (n=33648)
    Validation: 2024-04-21 00:00:00 -- 2024-04-21 23:00:00  (n=24)
Fold: 2
    Training:   2020-06-18 00:00:00 -- 2024-04-19 23:00:00  (n=33648)
    Validation: 2024-04-22 00:00:00 -- 2024-04-22 23:00:00  (n=24)
Fold: 3
    Training:   2020-06-18 00:00:00 -- 2024-04-19 23:00:00  (n=33648)
    Validation: 2024-04-23 00:00:00 -- 2024-04-23 23:00:00  (n=24)
Fold: 4
    Training:   2020-06-18 00:00:00 -- 2024-04-19 23:00:00  (n=33648)
    Validation: 2024-04-2

  0%|          | 0/60 [00:00<?, ?it/s]

Backtest error (MAE): 0.19748327859739462


In [8]:
# Plot predictions vs real value
# ======================================================================================
fig = go.Figure()
trace1 = go.Scatter(x=df_test.index, y=df_test['wave_height'], name="actual", mode="lines")
trace2 = go.Scatter(x=predictions.index, y=predictions['pred'], name="prediction", mode="lines")
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.update_layout(
    title="Actual value vs predicted in test data",
    xaxis_title="Date time",
    yaxis_title="Wave height (meters)",
    width=800,
    height=400,
    margin=dict(l=20, r=20, t=35, b=20),
    legend=dict(
        orientation="h",
        yanchor="top",
        y=1.1,
        xanchor="left",
        x=0.001
    )
)
fig.show()

### Hyperparameter tuning

The trained `ForecasterAutoreg` object used the first 24 lags and a `XGBRegressor` model with the default hyperparameters. However, there is no reason why these values are the most appropriate. To find the best hyperparameters, a Bayesian Search is performed using the `bayesian_search_forecaster()` function. The search is carried out using the same backtesting process as before, but each time, the model is trained with different combinations of hyperparameters and lags. It is important to note that the hiperparameter search must be done using the validation set, so the test data is never used.

In [23]:
# Hyperparameters search
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor = XGBRegressor(random_state=42),
                 lags      = 24, # This value will be replaced in the grid search
             )

# Lags used as predictors
# lags_grid = [24, 48, [1, 2, 3, 24]]

# Regressor hyperparameters search space
def search_space(trial):
    search_space  = {
        'n_estimators'  : trial.suggest_int('n_estimators', 600, 1200, step=50),
        'max_depth'     : trial.suggest_int('max_depth', 3, 12, step=1),
        'learning_rate' : trial.suggest_float('learning_rate', 0.01, 0.5),
        'reg_alpha'     : trial.suggest_float('reg_alpha', 0, 1, step=0.1),
        'reg_lambda'    : trial.suggest_float('reg_lambda', 0, 1, step=0.1),
        'lags'          : trial.suggest_categorical('lags', [24, 48, [1, 2, 3, 24]])
    } 
    return search_space

results_search, frozen_trial = bayesian_search_forecaster(
                                   forecaster         = forecaster,
                                   y                  = df.iloc[:end_val]['wave_height'],
                                   steps              = 24,
                                   metric             = 'mean_absolute_error',
                                   search_space       = search_space,
                                #    lags_grid          = lags_grid,
                                   initial_train_size = len(df[:end_train]),
                                   refit              = False,
                                   n_trials           = 120, # Increase for more exhaustive search
                                   random_state       = 42,
                                   return_best        = True,
                                   n_jobs             = 'auto',
                                   verbose            = False,
                                   show_progress      = True
                               )

/Users/alvarocorralescano/opt/anaconda3/envs/waves/lib/python3.11/site-packages/skforecast/model_selection/model_selection.py:1346: UserWarning:

The 'lags_grid' argument is deprecated and will be removed in a future version. Use the 'search_space' argument to define the candidate values for the lags. Example: {'lags' : trial.suggest_categorical('lags', [3, 5])}

Best trial: 92. Best value: 0.264842: 100%|██████████| 120/120 [06:28<00:00,  3.23s/it]


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24] 
  Parameters: {'n_estimators': 900, 'max_depth': 4, 'learning_rate': 0.010968332232162158, 'reg_alpha': 0.1, 'reg_lambda': 0.1}
  Backtesting metric: 0.2648422786047061



In [26]:
# Backtest final model on test data
# ==============================================================================
metric, predictions = backtesting_forecaster(
                          forecaster         = forecaster,
                          y                  = df['wave_height'],
                          steps              = 12,
                          metric             = 'mean_absolute_error',
                          initial_train_size = len(df[:end_val]),
                          refit              = False,
                          n_jobs             = 'auto',
                          verbose            = False, # Change to True to see detailed information
                          show_progress      = True
                      )

print(f"Backtest error: {metric:.2f}")

100%|██████████| 60/60 [00:00<00:00, 168.84it/s]

Backtest error: 0.13


No improvement over "dumb" XGBoost model

In [27]:
# Plot predictions vs real value
# ======================================================================================
fig = go.Figure()
trace1 = go.Scatter(x=df_test.index, y=df_test['wave_height'], name="actual", mode="lines")
trace2 = go.Scatter(x=predictions.index, y=predictions['pred'], name="prediction", mode="lines")
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.update_layout(
    title="Actual value vs predicted in test data",
    xaxis_title="Date time",
    yaxis_title="Wave height (meters)",
    width=800,
    height=400,
    margin=dict(l=20, r=20, t=35, b=20),
    legend=dict(
        orientation="h",
        yanchor="top",
        y=1.1,
        xanchor="left",
        x=0.001
    )
)
fig.show()